  - Cloud computation
      - Short running functional units
      - Long running computation
        - Managed kubernetes
        - RabbitMQ
        
        
          - Distributed computation
          - Dask
          - Spark - Large data processing and visualization with ImgLib2, N5, Spark, BigDataViewer, and Paintera https://www.janelia.org/sites/default/files/You%20%2B%20Janelia/Conferences/3.pdf
